In [ ]:
# import the required libraries
import os
import cPickle
import numpy as np
import xml.etree.ElementTree as ET
import random
import svgwrite
from IPython.display import SVG, display
from svg.path import Path, Line, Arc, CubicBezier, QuadraticBezier, parse_path
from svg.path import Path, Line, Arc, CubicBezier, QuadraticBezier
from svgpathtools import svg2paths, wsvg
np.set_printoptions(linewidth=np.nan, precision=3)
import cv2
import potrace
import matplotlib.pyplot as plt
from rdp import rdp
from thinning import guo_hall_thinning
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import networkx as nx
from tqdm import tqdm
import time
import codecs
import collections
import math
import json
import tensorflow as tf
from six.moves import xrange
# os.environ["CUDA_VISIBLE_DEVICES"]="1"
# libraries required for visualisation:
import svgwrite # conda install -c omnia svgwrite=1.1.6
import PIL
from PIL import Image
# set numpy output to something sensible
np.set_printoptions(precision=8, edgeitems=6, linewidth=200, suppress=True)
from sketch_rnn_class import *
from model_class import *
from utils_class import *
from rnn import *

In [ ]:
#HELPER FUNCTIONS
def draw_strokes(data, factor=0.2, svg_filename = '/tmp/sketch_rnn/svg/sample.svg'):
  tf.gfile.MakeDirs(os.path.dirname(svg_filename))
  min_x, max_x, min_y, max_y = get_bounds(data, factor)
  dims = (50 + max_x - min_x, 50 + max_y - min_y)
  dwg = svgwrite.Drawing(svg_filename, size=dims)
  dwg.add(dwg.rect(insert=(0, 0), size=dims,fill='white'))
  lift_pen = 1
  abs_x = 25 - min_x 
  abs_y = 25 - min_y
  p = "M%s,%s " % (abs_x, abs_y)
  command = "m"
  for i in xrange(len(data)):
    if (lift_pen == 1):
      command = "m"
    elif (command != "l"):
      command = "l"
    else:
      command = ""
    x = float(data[i,0])/factor
    y = float(data[i,1])/factor
    lift_pen = data[i, 2]
    p += command+str(x)+","+str(y)+" "
  the_color = "black"
  stroke_width = 1
  dwg.add(dwg.path(p).stroke(the_color,stroke_width).fill("none"))
  dwg.save()
  display(SVG(dwg.tostring()))

def get_bounds(data, factor=10):
  """Return bounds of data."""
  min_x = 0
  max_x = 0
  min_y = 0
  max_y = 0

  abs_x = 0
  abs_y = 0
  for i in range(len(data)):
    x = float(data[i, 0]) / factor
    y = float(data[i, 1]) / factor
    abs_x += x
    abs_y += y
    min_x = min(min_x, abs_x)
    min_y = min(min_y, abs_y)
    max_x = max(max_x, abs_x)
    max_y = max(max_y, abs_y)

  return (min_x, max_x, min_y, max_y)

def cubicbezier(x0, y0, x1, y1, x2, y2, x3, y3, n=20):
    # from http://rosettacode.org/wiki/Bitmap/B%C3%A9zier_curves/Cubic
    pts = []
    for i in range(n+1):
        t = float(i) / float(n)
        a = (1. - t)**3
        b = 3. * t * (1. - t)**2
        c = 3.0 * t**2 * (1.0 - t)
        d = t**3

        x = float(a * x0 + b * x1 + c * x2 + d * x3)
        y = float(a * y0 + b * y1 + c * y2 + d * y3)
        pts.append( (x, y) )
    return pts

def get_path_strings(svgfile):
    tree = ET.parse(svgfile)
    p = []
    for elem in tree.iter():
        if elem.attrib.has_key('d'):
            p.append(elem.attrib['d'])
    return p

def build_lines(svgfile, line_length_threshold = 10.0, min_points_per_path = 1, max_points_per_path = 3):
  # we don't draw lines less than line_length_threshold
  path_strings = get_path_strings(svgfile)
  lines = []

  for path_string in path_strings:
    full_path = parse_path(path_string)
    for i in range(len(full_path)):
      p = full_path[i]
      if type(p) != Line and type(p) != CubicBezier:
        print "encountered an element that is not just a line or bezier "
        print "type: ",type(p)
        print p
      else:
        x_start = p.start.real
        y_start = p.start.imag
        x_end = p.end.real
        y_end = p.end.imag
        line_length = np.sqrt((x_end-x_start)*(x_end-x_start)+(y_end-y_start)*(y_end-y_start))
        len_data.append(line_length)
        points = []
        if type(p) == CubicBezier:
          x_con1 = p.control1.real
          y_con1 = p.control1.imag
          x_con2 = p.control2.real
          y_con2 = p.control2.imag
          n_points = int(line_length / line_length_threshold)+1
          n_points = max(n_points, min_points_per_path)
          n_points = min(n_points, max_points_per_path)
          points = cubicbezier(x_start, y_start, x_con1, y_con1, x_con2, y_con2, x_end, y_end, n_points)
        else:
          points = [(x_start, y_start), (x_end, y_end)]
        if i == 0: # only append the starting point for svg
          lines.append([points[0][0], points[0][1], 0, 0]) # put eoc to be zero
        for j in range(1, len(points)):
          eos = 0
          if j == len(points)-1 and i == len(full_path)-1:
            eos = 1
          lines.append([points[j][0], points[j][1], eos, 0]) # put eoc to be zero
  lines = np.array(lines, dtype=np.float32)
      # make it relative moves
  lines[1:,0:2] -= lines[0:-1,0:2]
  lines[-1,3] = 1 # end of character
  lines[0] = [0, 0, 0, 0] # start at origin
  return lines[1:]

def lines_to_strokes(lines):
    eos = 0
    strokes = [[0, 0, 0]]
    for line in lines:
        print line
        linelen = len(line)
        for i in range(linelen):
            eos = 0 if i < linelen - 1 else 1
            strokes.append([line[i][0], line[i][1], eos])
    strokes = np.array(strokes)
    strokes[1:, 0:2] -= strokes[:-1, 0:2]
    return strokes[1:, :]

def encode(input_strokes):
  strokes = to_big_strokes(input_strokes,max_len=125).tolist()
  print(np.shape(strokes))
  strokes.insert(0, [0, 0, 1, 0, 0])
  seq_len = [len(input_strokes)]
  draw_strokes(to_normal_strokes(np.array(strokes)))
  return sess.run(eval_model.batch_z, feed_dict={eval_model.input_data: [strokes], eval_model.sequence_lengths: seq_len})[0]

def preprocess_single(stroke):
    scale_factor=0.15 #see model params
    stroke[:, 0:2] /= scale_factor
    return stroke

In [ ]:
#CONVERT FROM dx to POLYLINE format
all_lines = []
current_line = []
last_x = test_lines[0][0]
last_y = test_lines[0][1]
test_lines = test_lines[1:]
for row in test_lines:
    dx = row[0]
    dy = row[1]
    point = [last_x+dx,last_y+dy]
    if not(dx == 0 and dy == 0):
        current_line.append(point)
    if row[2] == 1 or row[3] == 1:
        print(np.shape(current_line))
        all_lines.append(current_line)
        current_line = []
    last_x = x
    last_y = y

In [ ]:
#CONVERT FROM X-Y TO SIMPLE POLYLINE
all_lines = []
current_line = []
for row in test_lines:
    x = row[0]
    y = row[1]
    point = [x,y]
    if not(x == 0 and y == 0):
        current_line.append(point)
    if row[2] == 1 or row[3] == 1:
        print(np.shape(current_line))
        all_lines.append(current_line)
        current_line = []
print np.shape(all_lines)
print np.shape(all_lines[0])

In [ ]:
#PERFORM RDP ON LINES
rdp_lines = []
for line in all_lines:
    print np.shape(line)
    new_line = rdp(line, epsilon = 0.2)
    print np.shape(new_line)
    rdp_lines.append(new_line)
print np.shape(rdp_lines)

In [ ]:
#LOAD CHECKPOINT
#load pathways
data_dir = 'datasets/'
models_root_dir = '' #same directory we're in now
model_dir = 'four'
[train_set, valid_set, test_set, hps_model, eval_hps_model, sample_hps_model] = load_env(data_dir, model_dir)

In [ ]:
# construct the sketch-rnn model here:
reset_graph()
model = Model(hps_model)
eval_model = Model(eval_hps_model, reuse=True)
sample_model = Model(sample_hps_model, reuse=True)
#run session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
# loads the weights from checkpoint into our model
load_checkpoint(sess, model_dir)

In [ ]:
#READ IN SINGLE SAMPLE AND PREDICT
data = np.load("datasets/nose.npz")
test_data = data['test']
stroke = np.random.choice(test_data)
stroke = np.array(stroke, dtype=np.float32)
#normalize stroke by dividing by normalizing scale factor
scale_factor = 45.8033
stroke[:, 0:2] /= scale_factor
draw_strokes(stroke)
prediction = predict_model(sess, eval_model, stroke, 125)
print(prediction)

In [ ]:
#CONVERT SINGLE SAMPLE & PREDICT
stroke = new_test_lines
stroke = np.array(stroke, dtype=np.float32)
#normalize stroke by dividing by normalizing scale factor
scale_factor = 10
stroke[:, 0:2] /= scale_factor
draw_strokes(stroke)
prediction = predict_model(sess, eval_model, stroke, 125)
print(prediction)

In [ ]:
#TEST ENTIRE BATCH - CONVERSION to CLASSIFICATION
data = np.load("datasets/nose.npz")
test_data = data['test']
preds = [0,0,0,0]
for i in range(0,100):
    stroke = np.random.choice(test_data)
    stroke = np.array(stroke, dtype=np.float32)
    #normalize stroke by dividing by normalizing scale factor
    scale_factor = 45.8033
    stroke[:, 0:2] /= scale_factor
    draw_strokes(stroke,svg_filename="features_to_stroke/test.svg")
    #read in SVG
    test_lines = build_lines("features_to_stroke/test.svg")
    #print test_lines
    new_test_lines = test_lines[:,:3]
    new_test_lines[1:, 0:2] -= new_test_lines[:-1, 0:2]
    #draw_strokes(new_test_lines,factor=0.2)
    stroke = new_test_lines
    stroke = np.array(stroke, dtype=np.float32)
    #normalize stroke by dividing by normalizing scale factor
    scale_factor = 10
    stroke[:, 0:2] /= scale_factor
    #draw_strokes(stroke,factor=0.02)
    prediction = int(predict_model(sess, eval_model, stroke, 125))
    preds[prediction] += 1
print(preds)
